# Model Type 2: Z-R point model

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.optim import SGD
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import mse_loss
from torch.nn.functional import mse_loss
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import time
import pandas as pd
from torch.utils.data import Dataset
import torch
from PIL import Image
from torchvision.transforms import ToTensor, Normalize, Compose
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import os
import pandas as pd
import numpy as np
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
import torch
from datetime import datetime, timedelta

# Local application/library specific imports
import importlib
import sys
import os
base_path = 'C:/Users/Henri/Emilie/Thesis'
sys.path.append(base_path + '/code')
import project_functions as pf
importlib.reload(pf) # Reloading the local module 

# Reloading the local module 
importlib.reload(pf)

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")
    
# Function to set seed for reproducibility
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
seed = 42
set_seed(seed)
    
base_path = 'C:/Users/Henri/Emilie/Thesis'

CUDA is available. Using GPU.


### Data loader

In [ ]:
class ZR_dataset(Dataset):
    def __init__(self, csv_file, image_dir='C:/Users/Henri/Emilie/Thesis/data/images/train', num_samples=100):
        self.df_full = pd.read_csv(csv_file)
        
        # Convert 'timestamp' column to datetime
        self.df_full['timestamp'] = pd.to_datetime(self.df_full['timestamp'])
        
        # Filter the rows where the minute value is a multiple of 10
        self.df_full = self.df_full[self.df_full['timestamp'].dt.minute % 10 == 0]
        
        # Filter out rows with Z = 0, R = null, or Z = null
        self.df_filtered = self.df_full[(self.df_full['R'] != 0) & (self.df_full['Z'] != 0) & ~self.df_full['R'].isnull() & ~self.df_full['Z'].isnull()]

        # Sample indices without replacement from the filtered DataFrame
        #if len(self.df_filtered) > num_samples:
        #    self.sampled_indices = np.random.choice(self.df_filtered.index, size=num_samples, replace=False)
        #else:
        self.sampled_indices = self.df_filtered.index

        # Create a sampled DataFrame based on the sampled indices
        self.df_sampled = self.df_filtered.loc[self.sampled_indices].reset_index(drop=True)

        # No sampling, use all data
        self.df_sampled = self.df_filtered.reset_index(drop=True)
        
        # Convert relevant columns to float and to a NumPy array for faster access
        self.features = self.df_sampled[['radar_norm', 'dist_coast', 'dist_radar', 'topo', 'toy_sin', 'toy_cos', 'tod_sin', 'tod_cos']].astype(float).values
        self.R = self.df_sampled['R'].astype(float).values
        self.Z = self.df_sampled['Z'].astype(float).values
        self.dBZ = self.df_sampled['dBZ'].astype(float).values
        self.image_paths = self.df_sampled['cropped_radar_image_path'].values  # Load image paths
        self.station_ids = self.df_sampled['stationId'].values  # Load station IDs
        self.image_dir = image_dir

In [ ]:
class ZR_dataset(Dataset):
    def __init__(self, csv_file, image_dir='C:/Users/Henri/Emilie/Thesis/data/images/train', num_samples=100):
        self.df_full = pd.read_csv(csv_file)
        
        # Convert 'timestamp' column to datetime
        self.df_full['timestamp'] = pd.to_datetime(self.df_full['timestamp'])
        
        # Filter the rows where the minute value is a multiple of 10
        self.df_full = self.df_full[self.df_full['timestamp'].dt.minute % 10 == 0]
        
        # Filter out rows with Z = 0, R = null, or Z = null
        self.df_filtered = self.df_full[(self.df_full['R'] != 0) & (self.df_full['Z'] != 0) & ~self.df_full['R'].isnull() & ~self.df_full['Z'].isnull()]

        # Sample indices without replacement from the filtered DataFrame
        #if len(self.df_filtered) > num_samples:
        #    self.sampled_indices = np.random.choice(self.df_filtered.index, size=num_samples, replace=False)
        #else:
        self.sampled_indices = self.df_filtered.index

        # Create a sampled DataFrame based on the sampled indices
        self.df_sampled = self.df_filtered.loc[self.sampled_indices].reset_index(drop=True)

        # No sampling, use all data
        self.df_sampled = self.df_filtered.reset_index(drop=True)
        
        # Convert relevant columns to float and to a NumPy array for faster access
        self.features = self.df_sampled[['radar_norm', 'dist_coast', 'dist_radar', 'topo', 'toy_sin', 'toy_cos', 'tod_sin', 'tod_cos']].astype(float).values
        self.R = self.df_sampled['R'].astype(float).values
        self.Z = self.df_sampled['Z'].astype(float).values
        self.dBZ = self.df_sampled['dBZ'].astype(float).values
        self.image_paths = self.df_sampled['cropped_radar_image_path'].values  # Load image paths
        self.station_ids = self.df_sampled['stationId'].values  # Load station IDs
        self.image_dir = image_dir

    def __len__(self):
        return len(self.R)  # Return the length of the sampled DataFrame

    def __getitem__(self, idx):
        # Directly access the pre-converted numpy arrays
        x = torch.tensor(self.features[idx], dtype=torch.float)
        R = torch.tensor(self.R[idx], dtype=torch.float)
        Z = torch.tensor(self.Z[idx], dtype=torch.float)
        dBZ = torch.tensor(self.dBZ[idx], dtype=torch.float)
        
        # Load the current image and its timestamp
        current_image_path = self.image_paths[idx]
        current_image = Image.open(current_image_path)
        current_image = ToTensor()(current_image)  # Converts the image to a PyTorch tensor and normalizes to [0, 1]

        # Parse the timestamp from the image filename
        timestamp_str = os.path.basename(current_image_path).split('_')[1] + '_' + os.path.basename(current_image_path).split('_')[2]
        try:
            timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d_%H-%M-%S')
        except ValueError:
            try:
                timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d')
            except ValueError as e:
                raise ValueError(f"Unexpected timestamp format in filename: {current_image_path}") from e
        
        # Get the station ID
        station_id = self.station_ids[idx]

        # Define time offsets including the current timestamp
        time_offsets = [
            timedelta(minutes=0),
            timedelta(minutes=10),
            timedelta(minutes=20),
            timedelta(minutes=30),
            timedelta(minutes=40),
            timedelta(minutes=50),
            timedelta(minutes=60)
        ]

        # Initialize a list to hold images and image paths
        images = []
        image_paths = []

        for offset in time_offsets:
            past_timestamp = timestamp - offset
            past_image_path = self.get_image_path(past_timestamp, station_id)
            #print(f'Loading image from: {past_image_path}')  # Debug print to check image paths
            image_paths.append(past_image_path)
            if past_image_path and os.path.exists(past_image_path):
                past_image = Image.open(past_image_path)
                past_image = ToTensor()(past_image)
                images.append(past_image)
            else:
                #print(f'Image not found, using blank image for: {past_image_path}')  # Debug print for missing images
                images.append(torch.zeros_like(current_image))  # Use a blank image if the past image is not found

        # Add the current image as the last channel again
        #images.append(current_image)
        #image_paths.append(current_image_path)

        # Stack images to form a multi-channel image
        multi_channel_image = torch.cat(images, dim=0)

        return {'R': R, 'Z': Z, 'dBZ': dBZ, 'x': x, 'image': multi_channel_image} #, 'image_paths': image_paths}

    def get_image_path(self, timestamp, station_id):
        # Convert timestamp to the string format used in filenames
        timestamp_str = timestamp.strftime('%Y-%m-%d_%H-%M-%S')
        # Construct the full path of the image
        path = os.path.join(self.image_dir, f'{timestamp.year}/{timestamp.month:02}/{timestamp.day:02}/{timestamp.hour:02}{timestamp.minute:02}', f'cropped_{timestamp_str}_{station_id}_63.png')
        #print(f'Constructed image path: {path}')  # Debug print to check constructed paths
        return path


In [ ]:
def crop_image(image_array, row, col, size):
    """
    Crop a square patch of size (size, size) from the image_array centered at (row, col).
    """
    half_size = size // 2
    cropped_image = image_array[
        row - half_size:row + half_size + 1,
        col - half_size:col + half_size + 1
    ]
    return cropped_image

def min_max_normalize(arr):
    return (arr - arr.min()) / (arr.max() - arr.min())


class ZR_dataset(Dataset):
    def __init__(self, csv_file, image_dir='C:/Users/Henri/Emilie/Thesis/data/images/train', num_samples=1000):
        self.df_full = pd.read_csv(csv_file)
        
        # Convert 'timestamp' column to datetime
        self.df_full['timestamp'] = pd.to_datetime(self.df_full['timestamp'])
        
        # Filter the rows where the minute value is a multiple of 10
        mask = (self.df_full['timestamp'].dt.minute % 10 == 0) & \
               (self.df_full['R'].notnull()) & (self.df_full['Z'].notnull()) & \
               (self.df_full['R'] != 0) & (self.df_full['Z'] != 0)
        self.df_filtered = self.df_full[mask]
        
        ## Sample indices without replacement from the filtered DataFrame
        #if len(self.df_filtered) > num_samples:
        #    self.sampled_indices = np.random.choice(self.df_filtered.index, size=num_samples, replace=False)
        #    self.df_sampled = self.df_filtered.loc[self.sampled_indices]
        #else:
        self.df_sampled = self.df_filtered
        
        # Reset index of the sampled DataFrame
        self.df_sampled.reset_index(drop=True, inplace=True)
        
        # Convert relevant columns to float and to a NumPy array for faster access
        self.features = self.df_sampled[['radar_norm', 'dist_coast', 'dist_radar', 'topo', 'toy_sin', 'toy_cos', 'tod_sin', 'tod_cos']].astype(float).values
        self.R = self.df_sampled['R'].astype(float).values
        self.Z = self.df_sampled['Z'].astype(float).values
        self.dBZ = self.df_sampled['dBZ'].astype(float).values
        self.image_paths = self.df_sampled['cropped_radar_image_path'].values  # Load image paths
        self.station_ids = self.df_sampled['stationId'].values  # Load station IDs
        self.image_dir = image_dir
        self.r_idx = self.df_sampled['r_idx'].astype(int).values  # Load row indices
        self.c_idx = self.df_sampled['c_idx'].astype(int).values  # Load column indices
        
        # Load the additional image arrays
        self.topo_image = np.load('topo_01.npy')
        self.dist_radar_image = np.load('dist_radar_01.npy')
        self.dist_coast_image = np.load('climate.npy')
        
        self.dist_coast_image = min_max_normalize(self.dist_coast_image)
        self.dist_radar_image = min_max_normalize(self.dist_radar_image)
        self.topo_image = min_max_normalize(self.topo_image)

        
        

    def __len__(self):
        return len(self.R)  # Return the length of the sampled DataFrame

    def __getitem__(self, idx):
        # Directly access the pre-converted numpy arrays
        x = torch.tensor(self.features[idx], dtype=torch.float)
        R = torch.tensor(self.R[idx], dtype=torch.float)
        Z = torch.tensor(self.Z[idx], dtype=torch.float)
        dBZ = torch.tensor(self.dBZ[idx], dtype=torch.float)
        
        # Load the current image and its timestamp
        current_image_path = self.image_paths[idx]
        current_image = Image.open(current_image_path)
        current_image = ToTensor()(current_image)  # Converts the image to a PyTorch tensor and normalizes to [0, 1]

        # Parse the timestamp from the image filename
        timestamp_str = os.path.basename(current_image_path).split('_')[1] + '_' + os.path.basename(current_image_path).split('_')[2]
        try:
            timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d_%H-%M-%S')
        except ValueError:
            try:
                timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d')
            except ValueError as e:
                raise ValueError(f"Unexpected timestamp format in filename: {current_image_path}") from e
        
        # Get the station ID
        station_id = self.station_ids[idx]

        # Define time offsets including the current timestamp
        time_offsets = [
            timedelta(minutes=0),
            timedelta(minutes=10),
            timedelta(minutes=20),
            timedelta(minutes=30),
            timedelta(minutes=40),
            timedelta(minutes=50),
            timedelta(minutes=60)
        ]

        # Initialize a list to hold images and image paths
        images = []
        image_paths = []

        for offset in time_offsets:
            past_timestamp = timestamp - offset
            past_image_path = self.get_image_path(past_timestamp, station_id)
            #print(f'Loading image from: {past_image_path}')  # Debug print to check image paths
            image_paths.append(past_image_path)
            if past_image_path and os.path.exists(past_image_path):
                past_image = Image.open(past_image_path)
                past_image = ToTensor()(past_image)
                images.append(past_image)
            else:
                #print(f'Image not found, using blank image for: {past_image_path}')  # Debug print for missing images
                images.append(torch.zeros_like(current_image))  # Use a blank image if the past image is not found

        # Add the current image as the last channel again
        # Crop additional image channels
        row = self.r_idx[idx]
        col = self.c_idx[idx]
        topo_crop = crop_image(self.topo_image, row, col, 63)
        dist_radar_crop = crop_image(self.dist_radar_image, row, col, 63)
        dist_coast_crop = crop_image(self.dist_coast_image, row, col, 63)
        
        # Convert these cropped images to tensors and normalize them
        topo_crop = ToTensor()(topo_crop).float()
        dist_radar_crop = ToTensor()(dist_radar_crop).float()
        dist_coast_crop = ToTensor()(dist_coast_crop).float()

        # Add the cropped images as additional channels
        images.append(topo_crop)
        images.append(dist_radar_crop)
        images.append(dist_coast_crop)

        # Stack images to form a multi-channel image
        multi_channel_image = torch.cat(images, dim=0)

        return {'R': R, 'Z': Z, 'dBZ': dBZ, 'x': x, 'image': multi_channel_image} #, 'image_paths': image_paths}

    def get_image_path(self, timestamp, station_id):
        # Convert timestamp to the string format used in filenames
        timestamp_str = timestamp.strftime('%Y-%m-%d_%H-%M-%S')
        # Construct the full path of the image
        path = os.path.join(self.image_dir, f'{timestamp.year}/{timestamp.month:02}/{timestamp.day:02}/{timestamp.hour:02}{timestamp.minute:02}', f'cropped_{timestamp_str}_{station_id}_63.png')
        #print(f'Constructed image path: {path}')  # Debug print to check constructed paths
        return path


In [ ]:
# Load data
BATCH_SIZE = 1024
train_dataset = ZR_dataset(csv_file=base_path + f'/data/trainZR_v3_63.csv')
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)#, num_workers=4, pin_memory=True)

validation_dataset = ZR_dataset(csv_file=base_path + f'/data/valZR_v3_63.csv')
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)#, num_workers=4, pin_memory=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader

# Function to plot radar images and print features along with image paths
def visualize_batch(batch, num_images=4):
    images = batch['image']
    features = batch['x']
    R = batch['R']
    Z = batch['Z']
    dBZ = batch['dBZ']
    #image_paths = batch['image_paths']
    
    num_images = min(num_images, len(images))
    num_channels = images.shape[1]

    fig, axs = plt.subplots(num_images, num_channels, figsize=(num_channels * 5, num_images * 5))

    # Ensure axs is a 2D array
    if num_images == 1:
        axs = np.expand_dims(axs, axis=0)
    if num_channels == 1:
        axs = np.expand_dims(axs, axis=1)

    for i in range(num_images):
        img = images[i].numpy()
        feature_str = ', '.join([f'{feature:.2f}' for feature in features[i].numpy()])

        for j in range(num_channels):
            ax = axs[i, j]
            ax.imshow(img[j, :, :], cmap='turbo')
            ax.axis('off')
            #image_path = image_paths[j][i]  # Get the corresponding image path
            title_path = 'None'#image_path[-32:]  # Extract the last 32 characters of the image path
            if j == 0:
                ax.set_title(f'0-lagged Image\n{title_path}\nFeatures: {feature_str}\n R: {R[i].item():.2f} | Z: {Z[i].item():.2f} | dBZ: {dBZ[i].item():.2f}')
            else:
                ax.set_title(f'{j*10}-lagged Image\n{title_path}')
                
            #print(np.isnan(img[j, :, :]).sum())

    plt.tight_layout()
    plt.show()

# Define paths
csv_file = base_path + '/data/trainZR_v3.csv'
image_dir = base_path + '/data/images/train'

# Get a batch from the DataLoader
for batch in train_dataloader:
    visualize_batch(batch, num_images=4)
    break  # Only visualize one batch


#### Architecture 

In [ ]:
class NN(nn.Module):
    def __init__(self, input_features=11, dropout_rate=0.1, cnn_dropout_rate=0.25):
        super(NN, self).__init__()

        # Image processing layers with added batch normalization and spatial dropout
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),  # Add batch normalization
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),  # Replace with spatial dropout

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),  # Add batch normalization
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),  # Replace with spatial dropout

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            #nn.BatchNorm2d(64),  # Add batch normalization
            nn.ReLU(),
            nn.Flatten()
        )

        # Compute the size of the flattened features after convolution and pooling
        reduced_image_size = 15  # From the final convolution layer with pooling
        total_image_features = 64 * (reduced_image_size ** 2)

        # Total features after concatenation
        concatenated_features = total_image_features + input_features

        # Define layers to learn A and B from features, including dropout
        self.to_A = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 1)  # Outputs A for given features
        )
        self.to_B = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 1)  # Outputs B for given features
        )

        # Adjusted to_R with Batch Normalization
        self.to_R = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            nn.BatchNorm1d(512),  # Add BatchNorm after Linear layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),  # Add BatchNorm after Linear layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            #nn.BatchNorm1d(128),  # Add BatchNorm after Linear layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 1)  # Outputs R for given features
        )

    def forward(self, x, image):
        image_features = self.conv_layers(image)
        # Concatenate image features with other features
        combined_features = torch.cat((image_features, x), dim=1)
        
        logA = F.softplus(self.to_A(combined_features))
        B = F.softplus(self.to_B(combined_features))
        logR = self.to_R(combined_features)
        
        return logA, B, logR


In [ ]:
class NN(nn.Module):
    def __init__(self, input_features=8, dropout_rate=0.1, cnn_dropout_rate=0.5, num_image_channels=11):
        super(NN, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(num_image_channels, 8, kernel_size=3, padding=1),
            #nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),

            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            #nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            #nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            #nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            #nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten()
        )

        self.fc_x = nn.Sequential(
            nn.Linear(input_features, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

        image_feature_dim = 128
        x_feature_dim = 128
        concatenated_features = image_feature_dim + x_feature_dim

        self.to_A = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 1),
            nn.Softplus()  # Ensure positive output
        )
        self.to_B = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 1),
            nn.Softplus()  # Ensure positive output
        )
        self.to_R = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            #nn.BatchNorm1d(128),
            #nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 1)
        )

    def forward(self, x, image):
        image_features = self.conv_layers(image)
        image_features = image_features.view(image_features.size(0), -1)

        x_features = self.fc_x(x)

        combined_features = torch.cat((image_features, x_features), dim=1)

        logA = self.to_A(combined_features)
        B = self.to_B(combined_features)
        logR = self.to_R(combined_features)

        return logA, B, logR


# Custom weight initialization
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)



In [ ]:
class NN(nn.Module):
    def __init__(self, input_features=11, dropout_rate=0.1, cnn_dropout_rate=0.25):
        super(NN, self).__init__()

        # Image processing layers with added batch normalization and spatial dropout
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),  # Add batch normalization
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),  # Replace with spatial dropout

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),  # Add batch normalization
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),  # Replace with spatial dropout

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            #nn.BatchNorm2d(64),  # Add batch normalization
            nn.ReLU(),
            nn.Flatten()
        )

        # Compute the size of the flattened features after convolution and pooling
        reduced_image_size = 15  # From the final convolution layer with pooling
        total_image_features = 64 * (reduced_image_size ** 2)

        # Total features after concatenation
        concatenated_features = total_image_features + input_features

        # Define layers to learn A and B from features, including dropout
        self.to_A = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 1)  # Outputs A for given features
        )
        self.to_B = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 1)  # Outputs B for given features
        )

        # Adjusted to_R with Batch Normalization
        self.to_R = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            nn.BatchNorm1d(512),  # Add BatchNorm after Linear layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),  # Add BatchNorm after Linear layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            #nn.BatchNorm1d(128),  # Add BatchNorm after Linear layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 1)  # Outputs R for given features
        )

    def forward(self, x, image):
        image_features = self.conv_layers(image)
        # Concatenate image features with other features
        combined_features = torch.cat((image_features, x), dim=1)
        
        logA = F.softplus(self.to_A(combined_features))
        B = F.softplus(self.to_B(combined_features))
        logR = self.to_R(combined_features)
        
        return logA, B, logR

In [ ]:
class NN(nn.Module):
    def __init__(self, input_features=8, dropout_rate=0.1, cnn_dropout_rate=0.1, num_image_channels=10):
        super(NN, self).__init__()

        # Image processing layers with added batch normalization and spatial dropout
        self.conv_layers = nn.Sequential(
            nn.Conv2d(num_image_channels, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),  # Add batch normalization
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),  # Replace with spatial dropout

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),  # Add batch normalization
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(cnn_dropout_rate),  # Replace with spatial dropout

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),  # Add batch normalization
            nn.ReLU(),
            nn.Flatten()
        )

        # Compute the size of the flattened features after convolution and pooling
        reduced_image_size = 15  # From the final convolution layer with pooling
        total_image_features = 64 * (reduced_image_size ** 2)

        # Total features after concatenation
        concatenated_features = total_image_features + input_features

        # Define layers to learn A and B from features, including dropout
        self.to_A = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 1)  # Outputs A for given features
        )
        self.to_B = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 1)  # Outputs B for given features
        )

        # Adjusted to_R with Batch Normalization
        self.to_R = nn.Sequential(
            nn.Linear(concatenated_features, 512),
            nn.BatchNorm1d(512),  # Add BatchNorm after Linear layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),  # Add BatchNorm after Linear layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 1)  # Outputs R for given features
        )

    def forward(self, x, image):
        image_features = self.conv_layers(image)
        # Concatenate image features with other features
        combined_features = torch.cat((image_features, x), dim=1)
        
        logA = F.softplus(self.to_A(combined_features))
        B = F.softplus(self.to_B(combined_features))
        logR = self.to_R(combined_features)
        
        return logA, B, logR

# Create the model
model = NN(input_features=8, dropout_rate=0.1, cnn_dropout_rate=0.1, num_image_channels=10)
model.to(device)

#### Loss

In [ ]:
def combined_loss(model, batch, device, alpha = 0.5):
    logZ = torch.log(batch['Z'].unsqueeze(1).to(device))
    logA, B, logR_pred = model(batch['x'].to(device), batch['image'].to(device))
    logR_true = torch.log(batch['R'].unsqueeze(1).to(device) * 60)  # Adjust units as necessary
    
    # Assuming A and B are constants for now, but they could be parameters of the model
    mse_loss = nn.MSELoss()
    data_loss = mse_loss(logR_pred, logR_true)
    physics_loss  = mse_loss(logR_pred, (logZ - logA)/B)
    total_loss = alpha * data_loss + (1 - alpha) * physics_loss
    return total_loss, data_loss, physics_loss


In [ ]:
def combined_loss(model, batch, device, alpha=0.5, beta=1.0, epsilon=1e-10):
    logZ = torch.log(batch['Z'].unsqueeze(1).to(device))
    logA, B, logR_pred = model(batch['x'].to(device), batch['image'].to(device))
    
    R_true = batch['R'].unsqueeze(1).to(device) * 60  # Adjust units as necessary
    logR_true = torch.log(R_true + epsilon)  # Use epsilon to avoid log(0)
    
    mse_loss = nn.MSELoss(reduction='none')
    data_loss = mse_loss(logR_pred, logR_true)
    
    physics_target = (logZ - logA) / B
    
    # Mask where R is zero for physics loss
    mask = (R_true > epsilon).float()
    physics_loss = mse_loss(logR_pred, physics_target) * mask
    
    # Weighting scheme to emphasize both small and large values
    weights = torch.exp(logR_true - torch.min(logR_true))  # Exponential for high values

    # Apply weights to data loss and physics loss
    weighted_data_loss = (data_loss * weights).mean()
    weighted_physics_loss = (physics_loss * weights).mean()
    
    total_loss = alpha * weighted_data_loss + (1 - alpha) * weighted_physics_loss
    return total_loss, weighted_data_loss, weighted_physics_loss


In [ ]:
def combined_loss(model, batch, device, alpha=0.5, beta=1.0):
    logZ = torch.log(batch['Z'].unsqueeze(1).to(device))
    logA, B, logR_pred = model(batch['x'].to(device), batch['image'].to(device))
    logR_true = torch.log(batch['R'].unsqueeze(1).to(device) * 60)  # Adjust units as necessary

    mse_loss = nn.MSELoss(reduction='none')
    data_loss = mse_loss(logR_pred, logR_true)
    physics_loss = mse_loss(logR_pred, (logZ - logA) / B)
    
    # Weight the data loss more for higher intensities
    weights = torch.exp(logR_true - torch.min(logR_true))  # Exponential to increase weight for high values
    weighted_data_loss = (data_loss * weights).mean()
    weighted_physics_loss = (physics_loss).mean()
    
    total_loss = alpha * weighted_data_loss + (1 - alpha) * weighted_physics_loss
    return total_loss, weighted_data_loss, weighted_physics_loss


#### Train Network

In [ ]:
from torch.optim import Adam

start_time = time.time() # Track the start time of the training process

# Adjustable hyperparameters
NUM_EPOCHS = 200 #40
LEARNING_RATE = 0.001 #0.001 #1e-06 No dropout
BATCH_SIZE = 1024
WEIGHT_DECAY = 0.001
ALPHA = 0.5
epsilon = 1e-6 
l1_lambda = 0.01

TYPE = 'PINN_CNN_Imagetopoint'
param_string = f"L1, E = {NUM_EPOCHS}, LR = {LEARNING_RATE}, BS = {BATCH_SIZE}, WD = {WEIGHT_DECAY}, alpha = {ALPHA}_"

# Define early stopping parameters
early_stopping_rounds = 10  # Number of epochs to wait for improvement
best_validation_loss = float('inf')
no_improvement_count = 0

# Check if CUDA is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# For plotting
train_losses = []
train_physics_losses = []
train_data_losses = []

validation_losses = []
validation_physics_losses = []
validation_data_losses = []


prev_avg_data_loss = None
prev_avg_physics_loss = None

# Optimizer - optimize A and B directly
#model = NN(input_features=8).to(device)
optimizer = Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)


# Example training loop
for epoch in range(NUM_EPOCHS):
    total_epoch_loss = 0
    data_epoch_loss = 0
    physics_epoch_loss = 0
    l1_epoch_loss = 0
    
    for batch in train_dataloader:
        batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
        optimizer.zero_grad() 
        total_loss, data_loss, physics_loss = combined_loss(model, batch, device, alpha = ALPHA)
        
        l1_penalty = sum(p.abs().sum() for p in model.parameters())
        
        # Total loss with L1 regularization
        loss_with_l1 = total_loss + l1_lambda * l1_penalty
        
        loss_with_l1.backward()
        optimizer.step()

        total_epoch_loss += total_loss.item()
        data_epoch_loss += data_loss.item()
        physics_epoch_loss += physics_loss.item()
        l1_epoch_loss += l1_penalty.item()

    # Calculate average loss over the epoch
    avg_epoch_loss = total_epoch_loss / len(train_dataloader)
    avg_data_loss = data_epoch_loss / len(train_dataloader)
    avg_physics_loss = physics_epoch_loss / len(train_dataloader)
    
    # Update previous losses for the next epoch
    prev_avg_data_loss = avg_data_loss
    prev_avg_physics_loss = avg_physics_loss

    train_losses.append(avg_epoch_loss)
    train_data_losses.append(avg_data_loss)
    train_physics_losses.append(avg_physics_loss)

    # Validation loop
    with torch.no_grad():  # No need to track gradients during validation
        total_validation_loss = 0
        validation_data_loss = 0
        validation_physics_loss = 0
        for batch in validation_dataloader:
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
            total_loss, data_loss, physics_loss = combined_loss(model, batch, device, alpha = ALPHA)

            total_validation_loss += total_loss.item()
            validation_data_loss += data_loss.item()
            validation_physics_loss += physics_loss.item()          

        avg_validation_loss = total_validation_loss / len(validation_dataloader)
        avg_validation_data_loss = validation_data_loss / len(validation_dataloader)
        avg_validation_physics_loss = validation_physics_loss / len(validation_dataloader)

        validation_losses.append(avg_validation_loss)
        validation_data_losses.append(avg_validation_data_loss)
        validation_physics_losses.append(avg_validation_physics_loss)
        
        scheduler.step(avg_validation_loss)
    
    if epoch % 1 == 0:
        print(f"Epoch {epoch + 1}, Loss: {avg_epoch_loss}, Validation Loss: {avg_validation_loss}, Alpha: {ALPHA}")
    
    # Check for early stopping
    if avg_validation_loss < best_validation_loss:
        best_validation_loss = avg_validation_loss
        no_improvement_count = 0
    else:
        no_improvement_count += 1
        if no_improvement_count >= early_stopping_rounds:
            print(f"Early stopping at epoch {epoch + 1} as validation loss didn't improve for {early_stopping_rounds} epochs.")
            break

end_time = time.time()
elapsed_time = end_time - start_time


In [ ]:
print(f"Training took {elapsed_time/60:.2f} min.")


In [ ]:
epochs = np.arange(epoch + 1)
fig, axs = plt.subplots(1, 3, figsize=(14, 4)) 

# Plot training and validation losses
axs[0].plot(epochs, train_losses, color = 'black', label = 'training loss')
axs[0].plot(epochs, validation_losses, color = 'darkorange', label = 'validation loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].set_title('Total loss')
#axs[0].set_ylim(0, 100)

# Plot training and validation losses
axs[1].plot(epochs, train_physics_losses, color = 'black', label = 'training loss')
axs[1].plot(epochs, validation_physics_losses, color = 'darkorange', label = 'validation loss')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')
axs[1].legend()
axs[1].set_title('Physics loss')
#axs[1].set_ylim(0, 100)

# Plot training and validation losses
axs[2].plot(epochs, train_data_losses, color = 'black', label = 'training loss')
axs[2].plot(epochs, validation_data_losses, color = 'darkorange', label = 'validation loss')
axs[2].set_xlabel('Epochs')
axs[2].set_ylabel('Loss')
axs[2].legend()
axs[2].set_title('Data loss')
#xs[2].set_ylim(0, 1)

plt.tight_layout()
save_path = param_string + 'model_conv2' + '.png'
plt.savefig(save_path, bbox_inches='tight', pad_inches=0.01, dpi=300)
plt.show()

#### Evaluatuion

In [ ]:
class ZR_test_dataset(Dataset):
    def __init__(self, csv_file, image_dir='C:/Users/Henri/Emilie/Thesis/data/images/test', num_samples=None):
        # Load the dataset
        df = pd.read_csv(csv_file)
        
        # Convert 'timestamp' column to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Filter the rows where the minute value is a multiple of 10
        df = df[df['timestamp'].dt.minute % 10 == 0]
        
        df = df[(df['R'] != 0) & (df['Z'] != 0) & ~df['R'].isnull() & ~df['Z'].isnull()]
        
        if num_samples is not None:
            df = df.sample(n=num_samples, random_state=42).reset_index(drop=True)
        
        self.sampled_indices = df.index

        # Create a sampled DataFrame based on the sampled indices
        self.df_sampled = df.loc[self.sampled_indices].reset_index(drop=True)

        # Convert relevant columns to float and to a NumPy array for faster access
        self.features = self.df_sampled[['radar_norm', 'dist_coast', 'dist_radar', 'topo', 'toy_sin', 'toy_cos', 'tod_sin', 'tod_cos']].astype(float).values
        self.R = self.df_sampled['R'].astype(float).values
        self.Z = self.df_sampled['Z'].astype(float).values
        self.dBZ = self.df_sampled['dBZ'].astype(float).values
        self.image_paths = self.df_sampled['cropped_radar_image_path'].values  # Load image paths
        self.station_ids = self.df_sampled['stationId'].values  # Load station IDs
        self.r_idx = self.df_sampled['r_idx'].astype(int).values  # Load row indices
        self.c_idx = self.df_sampled['c_idx'].astype(int).values  # Load column indices
        self.image_dir = image_dir

        # Load the additional image arrays
        self.topo_image = np.load('topo_01.npy')
        self.dist_radar_image = np.load('dist_radar_01.npy')
        self.dist_coast_image = np.load('climate.npy')

    def __len__(self):
        return len(self.R)

    def __getitem__(self, idx):
        # Directly access the pre-converted numpy arrays
        x = torch.tensor(self.features[idx], dtype=torch.float)
        R = torch.tensor(self.R[idx], dtype=torch.float)
        Z = torch.tensor(self.Z[idx], dtype=torch.float)
        dBZ = torch.tensor(self.dBZ[idx], dtype=torch.float)

        # Load the current image and its timestamp
        current_image_path = self.image_paths[idx]
        current_image = Image.open(current_image_path)
        current_image = ToTensor()(current_image)  # Converts the image to a PyTorch tensor and normalizes to [0, 1]

        # Parse the timestamp from the image filename
        timestamp_str = os.path.basename(current_image_path).split('_')[1] + '_' + os.path.basename(current_image_path).split('_')[2]
        try:
            timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d_%H-%M-%S')
        except ValueError:
            try:
                timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d')
            except ValueError as e:
                raise ValueError(f"Unexpected timestamp format in filename: {current_image_path}") from e

        # Get the station ID
        station_id = self.station_ids[idx]

        # Define time offsets including the current timestamp
        time_offsets = [
            timedelta(minutes=0),
            timedelta(minutes=10),
            timedelta(minutes=20),
            timedelta(minutes=30),
            timedelta(minutes=40),
            timedelta(minutes=50),
            timedelta(minutes=60)
        ]

        # Initialize a list to hold images
        images = []

        for offset in time_offsets:
            past_timestamp = timestamp - offset
            past_image_path = self.get_image_path(past_timestamp, station_id)
            if past_image_path and os.path.exists(past_image_path):
                past_image = Image.open(past_image_path)
                past_image = ToTensor()(past_image)
                images.append(past_image)
            else:
                images.append(torch.zeros_like(current_image))  # Use a blank image if the past image is not found

        # Crop additional image channels
        row = self.r_idx[idx]
        col = self.c_idx[idx]
        topo_crop = crop_image(self.topo_image, row, col, 63)
        dist_radar_crop = crop_image(self.dist_radar_image, row, col, 63)
        dist_coast_crop = crop_image(self.dist_coast_image, row, col, 63)

        # Convert these cropped images to tensors and normalize them
        topo_crop = ToTensor()(topo_crop).float()
        dist_radar_crop = ToTensor()(dist_radar_crop).float()
        dist_coast_crop = ToTensor()(dist_coast_crop).float()

        # Add the cropped images as additional channels
        images.append(topo_crop)
        images.append(dist_radar_crop)
        images.append(dist_coast_crop)

        # Stack images to form a multi-channel image
        multi_channel_image = torch.cat(images, dim=0)

        return {'R': R, 'Z': Z, 'dBZ': dBZ, 'x': x, 'image': multi_channel_image}

    def get_image_path(self, timestamp, station_id):
        # Convert timestamp to the string format used in filenames
        timestamp_str = timestamp.strftime('%Y-%m-%d_%H-%M-%S')
        # Construct the full path of the image
        path = os.path.join(self.image_dir, f'{timestamp.year}/{timestamp.month:02}/{timestamp.day:02}/{timestamp.hour:02}{timestamp.minute:02}', f'cropped_{timestamp_str}_{station_id}_63.png')
        return path


In [ ]:
# Load test data
test_dataset = ZR_test_dataset(csv_file=base_path + f'/data/testZR_v3_63.csv')
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model.eval()  # Set the model to evaluation mode

logAs = []  
Bs = []
logR_preds = []

with torch.no_grad(): 
    for batch in tqdm(test_dataloader):
        features = batch['x'].to(device)
        images = batch['image'].to(device)

        # Make predictions
        logA, B, x = model(features, images)

        # Collect predictions
        logAs.append(logA)
        Bs.append(B)
        logR_preds.append(x)

In [ ]:
# Load test data
df_test = pd.read_csv(base_path + f'/data/testZR_v3_63.csv')
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test = df_test[df_test['timestamp'].dt.minute % 10 == 0]

df_test.head()

# Convert to appropriate types or calculate fields for entire DataFrame
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test['year'] = df_test['timestamp'].dt.year
df_test['month'] = df_test['timestamp'].dt.month

# Temporary fields to handle calculations (these can be adjusted later as needed)
df_test['R_true'] = np.array(df_test['R'] * 60)  # Convert R to the right unit for the whole DataFrame

# Identify rows where R and Z are not zero or NaN
valid_idx = df_test[(df_test['R'] != 0) & (df_test['Z'] != 0) & ~df_test['R'].isnull() & ~df_test['Z'].isnull()].index

# Apply calculations only for valid rows
df_test[['MP', 'MP_opt', 'R_pred_R', 'R_pred_AB']] = 0.0
df_test.loc[valid_idx, 'MP'] = np.array(pf.dbz_to_R_marshall_palmer(df_test.loc[valid_idx, 'dBZ'], 200, 1.6)).astype(float)
A_est = 196.0
B_est = 3.4
df_test.loc[valid_idx, 'MP_opt'] = np.array(pf.dbz_to_R_marshall_palmer(df_test.loc[valid_idx, 'dBZ'], A_est, B_est)).astype(float)
df_test.loc[valid_idx, 'A'] = np.exp(torch.cat(logAs).cpu().numpy())
df_test.loc[valid_idx, 'B'] = torch.cat(Bs).cpu().numpy()
df_test.loc[valid_idx, 'R_pred_R'] = np.exp(torch.cat(logR_preds).cpu().numpy())
df_test.loc[valid_idx, 'R_pred_AB'] = np.array(pf.dbz_to_R_marshall_palmer(df_test.loc[valid_idx, 'dBZ'], df_test.loc[valid_idx, 'A'], df_test.loc[valid_idx, 'B']))

# Calculate errors only for valid rows
df_test['error_standard'] = df_test['R_true'] - df_test['MP']
df_test['error_pred_R'] = df_test['R_true'] - df_test['R_pred_R']
df_test['error_pred_AB'] = df_test['R_true'] - df_test['R_pred_AB']

In [ ]:
(torch.cat(logR_preds).cpu().numpy())

In [ ]:
fig_string ='test'

# Example 1
pf.plot_hyetograph_m2(df_test, station_id=5049, 
               start_timestamp='2023-04-12 20:00:00', 
               end_timestamp='2023-04-13 08:30:00', 
               A_est=A_est, B_est=B_est, save_path = fig_string,
               tick_interval=6)  

# Example 2
pf.plot_hyetograph_m2(df_test, station_id=5195,
               start_timestamp='2023-07-26 09:00:00', 
               end_timestamp='2023-07-26 16:00:00', 
               A_est=A_est, B_est=B_est, save_path = fig_string,
               tick_interval=6)  

# Example 3
pf.plot_hyetograph_m2(df_test, station_id=5049,
               start_timestamp='2023-08-24 15:00:00', 
               end_timestamp='2023-08-25 08:00:00', 
               A_est=A_est, B_est=B_est, save_path = fig_string,
               tick_interval=6)  

# Example 4 
pf.plot_hyetograph_m2(df_test, station_id=5340,
               start_timestamp='2023-10-02 20:00:00', 
               end_timestamp='2023-10-03 11:00:00', 
               A_est=A_est, B_est=B_est, save_path = fig_string,
               tick_interval=6)  

# Example 5 
pf.plot_hyetograph_m2(df_test, station_id=5340,
               start_timestamp='2023-12-21 00:00:00', 
               end_timestamp='2023-12-21 11:00:00', 
               A_est=A_est, B_est=B_est, save_path = fig_string,
               tick_interval=6)  

# Example 6 
pf.plot_hyetograph_m2(df_test, station_id=5732,
               start_timestamp='2023-04-11 00:00:00', 
               end_timestamp='2023-04-11 18:00:00', 
               A_est=A_est, B_est=B_est, save_path = fig_string,
               tick_interval=6)  

# Example 7 
pf.plot_hyetograph_m2(df_test, station_id=5461,
               start_timestamp='2023-05-05 20:00:00', 
               end_timestamp='2023-05-06 09:00:00', 
               A_est=A_est, B_est=B_est, save_path = fig_string,
               tick_interval=6)  

In [ ]:
# Drop rows where any of the specified columns have NaN values
df_clean =  df_test.dropna(subset=['R_true', 'MP', 'R_pred_R', 'R_pred_AB', 'A', 'B'])

metrics_MP = pf.evaluation_metrics(df_clean['R_true'], df_clean['MP'])
metrics_MP_opt = pf.evaluation_metrics(df_clean['R_true'], df_clean['MP_opt'])
metrics_R_pred_R = pf.evaluation_metrics(df_clean['R_true'], df_clean['R_pred_R'])
metrics_R_pred_AB = pf.evaluation_metrics(df_clean['R_true'], df_clean['R_pred_AB'])

# Create a DataFrame from the metrics
metrics_df = pd.DataFrame({
    'MP': metrics_MP,
    'MP_opt': metrics_MP_opt,
    'R_pred_R': metrics_R_pred_R,
    'R_pred_AB': metrics_R_pred_AB
})

# Transpose the DataFrame to have models as rows and metrics as columns
metrics_df = metrics_df.T
metrics_df

In [ ]:
from pandas.plotting import table

# Create a figure and a single subplot
fig, ax = plt.subplots(figsize=(8, 3))  # Adjust the figure size as needed

# Hide the axes
ax.axis('off')

# Add a table at the bottom of the axes
tbl = table(ax, metrics_df, loc='center')

# Save the figure as an image file
save_path = param_string + 'metrics' + '.png'
plt.savefig(save_path, bbox_inches='tight', dpi=300)  # Adjust DPI as needed

plt.show()
# Optionally: close the figure
#plt.close(fig)

In [ ]:
print('Total absolute error MP_opt:', abs(df_test['R_true'].loc[valid_idx] / 6  -  df_test['MP_opt'].loc[valid_idx] / 6).sum())
print('Total absolute error R:', abs(df_test['R_true'].loc[valid_idx] / 6  -  df_test['R_pred_R'].loc[valid_idx] / 6).sum())
print('Total absolute error AB:', abs(df_test['R_true'].loc[valid_idx] / 6  -  df_test['R_pred_AB'].loc[valid_idx] / 6).sum())


In [ ]:
# by Rain Intensity Group
pf.plot_goruped_rain_intensity_error_m2(df_test.loc[valid_idx,:].copy(), TYPE = 'M3: Image-to-point PINN', save_path = fig_string)

In [ ]:
# Plot histograms of A and B
fig, axs = plt.subplots(2, 4, figsize=(16, 6))

axs = axs.flatten()

axs[0].hist(df_test['A'][np.isfinite(df_test['A'])], color='black', alpha=0.7)
#axs[0].set_title('Histogram of A')
axs[0].set_xlabel('A')
axs[0].set_ylabel('Frequency')

axs[4].hist(df_test['B'], bins=50, color='black', alpha=0.7)
#axs[4].set_title('Histogram of B')
axs[4].set_xlabel('B')
axs[4].set_ylabel('Frequency')

flierprops = dict(marker='.', markerfacecolor='black', markersize=2, linestyle='none')

groups = df_clean.groupby('month')['A'].apply(list)
data_for_boxplot = [group for name, group in groups.items()]
axs[1].boxplot(data_for_boxplot, flierprops=flierprops)
#axs[1].set_title('Boxplot of A by month')
axs[1].set_xlabel('Month')
axs[1].set_ylabel('A')

groups = df_clean.groupby('month')['B'].apply(list)
data_for_boxplot = [group for name, group in groups.items()]
axs[5].boxplot(data_for_boxplot, flierprops=flierprops)
#axs[5].set_title('Boxplot of B by month')
axs[5].set_xlabel('Month')
axs[5].set_ylabel('B')

groups = df_clean.groupby('stationId')['A'].apply(list)
data_for_boxplot = [group for name, group in groups.items()]
station_ids = [name for name, group in groups.items()]
axs[2].boxplot(data_for_boxplot, flierprops=flierprops,  labels=station_ids)
#axs[2].set_title('Boxplot of A by station')
axs[2].set_xlabel('Station ID')
axs[2].set_xticklabels(station_ids, rotation=45) 
axs[2].set_ylabel('A')

groups = df_clean.groupby('stationId')['B'].apply(list)
data_for_boxplot = [group for name, group in groups.items()]
axs[6].boxplot(data_for_boxplot, flierprops=flierprops,  labels=station_ids)
#axs[6].set_title('Boxplot of B by station')
axs[6].set_xlabel('Station ID')
axs[6].set_xticklabels(station_ids, rotation=45) 
axs[6].set_ylabel('B')

axs[3].scatter(df_clean['dBZ'], df_clean['A'], color='black', alpha=0.1, s= 1)
#axs[3].set_title('Scatter plot of A and dBZ')
axs[3].set_xlabel('dBZ')
axs[3].set_ylabel('A')

axs[7].scatter(df_clean['dBZ'], df_clean['B'], color='black', alpha=0.1, s= 1)
#axs[7].set_title('Scatter plot of B and dBZ')
axs[7].set_xlabel('dBZ')


plt.tight_layout()

save_path = param_string + 'A_B' + '.png'
plt.savefig(save_path, bbox_inches='tight', pad_inches=0.01, dpi=300)

plt.show()

In [ ]:
df_filtered = df_test[df_test['stationId'] == 5049]
df_filtered = df_filtered.reset_index(drop=True)

R_pred_R = df_filtered['R_pred_R']
R_pred_AB = df_filtered['R_pred_AB']
R_true = df_filtered['R_true']
MP = df_filtered['MP']
MP_opt = df_filtered['MP_opt']

idx = np.linspace(3120, 3270, 251, dtype=int)

df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])
t = df_filtered['timestamp'][idx]

plt.figure(figsize=(14,6))
plt.subplot(2,1,1)
plt.plot(t, R_true[idx], label = 'Gauge', lw = 2, color = 'black')
plt.plot(t, MP[idx], '--', label = 'Marshall-Palmer A=200, B=1.6', color = 'darkorange')
plt.plot(t, R_pred_R[idx], '-', label = f'NN R', color = 'purple')
#plt.plot(t, R_pred_AB[idx], '-', label = f'NN AB', color = 'dodgerblue')
plt.plot(t, MP_opt[idx], '--', label = f'Marshall-Palmer A={A_est:0.4}, B={B_est:0.2}', color = 'green')
plt.ylabel('Rain rate (mm/h)')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.legend()

error_standard = R_true - MP
error_R = R_true - R_pred_R
error_AB = R_true - R_pred_AB
error_MP_opt = R_true - MP_opt

plt.subplot(2,1,2)
#plt.plot(t, abs(error_standard[idx]), color = 'darkorange', label = 'Marshall-Palmer A=200, B=1.6')
plt.plot(t, abs(error_R[idx]), color = 'purple', label = f'NN (R)')
plt.plot(t, abs(error_AB[idx]), color = 'dodgerblue', label = f'NN (AB)')
plt.plot(t, abs(error_MP_opt[idx]), '--', color = 'green', label = f'Marshall-Palmer A={A_est:0.4}, B={B_est:0.2}')
plt.xlabel('Time')
plt.ylabel('Absolute error')
plt.legend()

plt.tight_layout()
save_path = param_string + 'model2_ts'  + '.png'
plt.savefig(save_path, bbox_inches='tight', pad_inches=0.01, dpi=300)
plt.show()

In [ ]:
df_filtered = df_test[df_test['stationId'] == 5049]
df_filtered = df_filtered.reset_index(drop=True)

R_pred_R = df_filtered['R_pred_R']
R_pred_AB = df_filtered['R_pred_AB']
R_true = df_filtered['R_true']
MP = df_filtered['MP']
MP_opt = df_filtered['MP_opt']

#idx = np.linspace(1150, 1300, 150, dtype=int)
idx = np.linspace(6240, 6500, 260, dtype=int)
df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])
t = df_filtered['timestamp'][idx]

plt.figure(figsize=(16,6))
plt.subplot(2,1,1)
plt.plot(t, R_true[idx], label = 'Gauge', lw = 3, color = 'black')
#plt.plot(t, MP[idx], '--', label = 'Marshall-Palmer A=200, B=1.6', color = 'darkorange')
plt.plot(t, R_pred_R[idx], '-', label = f'M2: R', color = 'purple')
plt.plot(t, R_pred_AB[idx], '-', label = f'M2: AB', color = 'dodgerblue')
plt.plot(t, MP_opt[idx], '--', label = f'M1: Marshall-Palmer A={A_est:0.4}, B={B_est:0.2}', color = 'green')
plt.ylabel('Rain rate (mm/h)')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.legend()

error_standard = R_true - MP
error_R = R_true - R_pred_R
error_AB = R_true - R_pred_AB
error_MP_opt = R_true - MP_opt

plt.subplot(2,1,2)
#plt.plot(t, abs(error_standard[idx]), color = 'darkorange', label = 'Marshall-Palmer A=200, B=1.6')
plt.plot(t, abs(error_R[idx]), color = 'purple', label = f'M2: R')
plt.plot(t, abs(error_AB[idx]), color = 'dodgerblue', label = f'M2: AB')
plt.plot(t, abs(error_MP_opt[idx]), '--', color = 'green', label = f'M1: Marshall-Palmer A={A_est:0.4}, B={B_est:0.2}')
plt.xlabel('Time')
plt.ylabel('Absolute error')
plt.legend()

plt.tight_layout()
save_path = param_string + 'model2_ts'  + '.png'
plt.savefig(save_path, bbox_inches='tight', pad_inches=0.01, dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Assuming df_test is already defined and contains the necessary data

TYPE = 'M3: Image-to-point PINN'
# Group by 'stationId' and compute cumulative sums for each group
grouped = df_test.groupby('stationId')

# Initialize an empty dictionary to store cumulative sums
cumulative_sums = {'timestamp': [], 'R_true': [], 'MP': [], 'MP_opt': [], 'R_pred_R': [], 'R_pred_AB': []}

# Loop over each group and compute the cumulative sums
for name, group in grouped:
    group = group.sort_values('timestamp')  # Ensure the group is sorted by timestamp

    # Remove rows with NaN values before computing cumulative sums
    group = group.dropna(subset=['R_true', 'MP', 'MP_opt', 'R_pred_R', 'R_pred_AB'])

    # Calculate cumulative sums
    cumulative_sums['timestamp'].append(group['timestamp'].values)
    cumulative_sums['R_true'].append(np.cumsum(group['R_true'].values))
    cumulative_sums['MP'].append(np.cumsum(group['MP'].values))
    cumulative_sums['MP_opt'].append(np.cumsum(group['MP_opt'].values))
    cumulative_sums['R_pred_R'].append(np.cumsum(group['R_pred_R'].values))
    cumulative_sums['R_pred_AB'].append(np.cumsum(group['R_pred_AB'].values))

# Find the minimum length of all cumulative sum arrays
min_length = min(len(arr) for arr in cumulative_sums['R_true'])

# Truncate all arrays to the minimum length
cumulative_sums['timestamp'] = [arr[:min_length] for arr in cumulative_sums['timestamp']]
cumulative_sums['R_true'] = np.array([arr[:min_length] for arr in cumulative_sums['R_true']])
cumulative_sums['MP'] = np.array([arr[:min_length] for arr in cumulative_sums['MP']])
cumulative_sums['MP_opt'] = np.array([arr[:min_length] for arr in cumulative_sums['MP_opt']])
cumulative_sums['R_pred_R'] = np.array([arr[:min_length] for arr in cumulative_sums['R_pred_R']])
cumulative_sums['R_pred_AB'] = np.array([arr[:min_length] for arr in cumulative_sums['R_pred_AB']])

# Compute the average cumulative sums across all stationIds
avg_R_true = np.mean(cumulative_sums['R_true'], axis=0)
avg_MP = np.mean(cumulative_sums['MP'], axis=0)
avg_MP_opt = np.mean(cumulative_sums['MP_opt'], axis=0)
avg_R_pred_R = np.mean(cumulative_sums['R_pred_R'], axis=0)
avg_R_pred_AB = np.mean(cumulative_sums['R_pred_AB'], axis=0)

# Handle timestamps separately
# Convert timestamps to numerical values (e.g., Unix timestamps)
timestamps_numeric = [ts[:min_length].astype(np.int64) for ts in cumulative_sums['timestamp']]
timestamps_numeric = np.array(timestamps_numeric)
avg_timestamp_numeric = np.mean(timestamps_numeric, axis=0)
avg_timestamp = pd.to_datetime(avg_timestamp_numeric)

# Create a figure and a set of subplots
fig, axs = plt.subplots(1, 1, figsize=(12, 8))  # 1 Row, 1 Column

# Plot the average cumulative sums
axs.plot(avg_timestamp, avg_R_true, label='True Values', color='black', lw=3)
axs.plot(avg_timestamp, avg_MP, label='M0: Standard Parameters', color = 'orangered', alpha = 0.8)
axs.plot(avg_timestamp, avg_MP_opt, label='M1: Optimized Parameters', color = 'green', alpha = 0.8)
axs.plot(avg_timestamp, avg_R_pred_R, label=f'{TYPE} (R)', color = 'blue', alpha = 0.8)
axs.plot(avg_timestamp, avg_R_pred_AB, label=f'{TYPE} (AB)', color = 'purple', alpha = 0.8)
axs.legend()
axs.set_ylabel('mm/h')
axs.set_title('Average Cumulative True and Predicted Values')

plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Assuming df_test is already defined and contains the necessary data

# Get unique station IDs
station_ids = df_test['stationId'].unique()

# Create a 4x4 grid of subplots
fig, axs = plt.subplots(4, 4, figsize=(20, 16))  # 4 Rows, 4 Columns

# Loop over each stationId and plot the cumulative values
for i, station_id in enumerate(station_ids):
    # Filter data for the current stationId
    df_filtered = df_test[df_test['stationId'] == station_id].sort_values('timestamp')
    
    # Remove rows with NaN values
    df_filtered = df_filtered.dropna(subset=['R_true', 'MP', 'MP_opt', 'R_pred_R', 'R_pred_AB'])
    
    # Compute cumulative sums
    cumsum_R_true = np.cumsum(df_filtered['R_true'].values)
    cumsum_MP = np.cumsum(df_filtered['MP'].values)
    cumsum_MP_opt = np.cumsum(df_filtered['MP_opt'].values)
    cumsum_R_pred_R = np.cumsum(df_filtered['R_pred_R'].values)
    cumsum_R_pred_AB = np.cumsum(df_filtered['R_pred_AB'].values)
    
    # Get the subplot axes
    ax = axs[i // 4, i % 4]
    
    # Plot cumulative sums
    ax.plot(df_filtered['timestamp'], cumsum_R_true, label='True Values', color='black', lw=3)
    ax.plot(df_filtered['timestamp'], cumsum_MP, label='M0: Standard Parameters', color = 'orangered', alpha = 0.8)
    ax.plot(df_filtered['timestamp'], cumsum_MP_opt, label='M1: Optimized Parameters', color = 'green', alpha = 0.8)
    ax.plot(df_filtered['timestamp'], cumsum_R_pred_R, label='M3: Image-to-point PINN (R)', color = 'blue', alpha = 0.8)
    ax.plot(df_filtered['timestamp'], cumsum_R_pred_AB, label='M3: Image-to-point PINN (AB)', color = 'purple', alpha = 0.8)
    
    # Set plot title
    ax.set_title(f'Station ID: {station_id}')
    
    # Set labels only on the leftmost column and bottom row
    if i % 4 == 0:
        ax.set_ylabel('mm/h')
    if i // 4 == 3:
        ax.set_xlabel('Date')
        ax.set_xticklabels(ax.get_xticks(), rotation=45)
    else:
        ax.set_xticklabels([])
    
    # Only add legend to the first subplot
    if i == 0:
        ax.legend()

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
metricsMP = pf.calculate_metrics2(df_test, 'MP')
metricsMP_opt = pf.calculate_metrics2(df_test, 'MP_opt')
metricsR = pf.calculate_metrics2(df_test, 'R_pred_R')
metricsAB = pf.calculate_metrics2(df_test, 'R_pred_AB')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_error_metrics(metrics_df_MP, metrics_df_MP_opt, metrics_df_R, metrics_df_AB, save_path=None):
    error_metrics = ['RMSE', 'MAE', 'MSE', 'ME']
    aggregation_levels = ['30T', '1H', '1D']
    aggregation_levels_str = ['30 min', '1 hour', '1 day']
    
    num_rows = len(aggregation_levels) + 1  # +1 for the non-aggregated metrics
    num_cols = len(error_metrics)
    
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(35, 20))
    
    colors = {
        'MP': 'orangered',
        'MP_opt': 'green',
        'R': 'blue',
        'AB': 'purple'
    }
    labels = {
        'MP': 'M0: Standard Parameters',
        'MP_opt': 'M1: Optimized Parameters',
        'R': 'M3: Image-to-point PINN (R)',
        'AB': 'M3: Image-to-point PINN (AB)'
    }
    
    for col, metric in enumerate(error_metrics):
        # Top row: non-aggregated metrics
        ax = axs[0, col]
        positions = np.arange(len(metrics_df_MP['StationId']))
        width = 0.2  # Width of the bars
        ax.bar(positions - 1.5 * width, metrics_df_MP[metric], width, label=labels['MP'], color=colors['MP'], alpha=0.7)
        ax.bar(positions - 0.5 * width, metrics_df_MP_opt[metric], width, label=labels['MP_opt'], color=colors['MP_opt'], alpha=0.7)
        ax.bar(positions + 0.5 * width, metrics_df_R[metric], width, label=labels['R'], color=colors['R'], alpha=0.7)
        #ax.bar(positions + 1.5 * width, metrics_df_AB[metric], width, label=labels['AB'], color=colors['AB'], alpha=0.7)
        ax.set_title(f'{metric}', size=18)
        ax.set_xticks([])
        ax.set_xticklabels([])
        if col == 0:
            ax.set_ylabel('5 min', fontsize=18)

        # Subsequent rows: aggregated metrics
        for row, interval in enumerate(aggregation_levels, start=1):
            ax = axs[row, col]
            interval_col = f'{metric}_{interval}'
            ax.bar(positions - 1.5 * width, metrics_df_MP[interval_col], width, label=labels['MP'], color=colors['MP'], alpha=0.7)
            ax.bar(positions - 0.5 * width, metrics_df_MP_opt[interval_col], width, label=labels['MP_opt'], color=colors['MP_opt'], alpha=0.7)
            ax.bar(positions + 0.5 * width, metrics_df_R[interval_col], width, label=labels['R'], color=colors['R'], alpha=0.7)
            #ax.bar(positions + 1.5 * width, metrics_df_AB[interval_col], width, label=labels['AB'], color=colors['AB'], alpha=0.7)
            
            if row == 0:
                ax.set_title(f'{metric}', fontsize=14)
            if col == 0:
                ax.set_ylabel(f'{aggregation_levels_str[row-1]}', fontsize=18)
            if row == num_rows - 1:
                ax.set_xticks(positions)
                ax.set_xticklabels(metrics_df_MP['StationId'], rotation=45, fontsize=16)
            else:
                ax.set_xticks([])
                ax.set_xticklabels([])

    # Create a single legend for the entire figure
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.99), ncol=2, fontsize=18, prop={'size': 18})
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)  # Adjust the top to make space for the legend
    
    if save_path:        
        save_path = f'{save_path}_error_aggregated.png'
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0.01, dpi=300)
        
    plt.show()

# Example usage:
# metricsMP = calculate_metrics(df_test, 'MP')
# metricsMP_opt = calculate_metrics(df_test, 'MP_opt')
# metricsR = calculate_metrics(df_test, 'R_pred_R')
# metricsAB = calculate_metrics(df_test, 'R_pred_AB')

plot_error_metrics(metricsMP, metricsMP_opt, metricsR, metricsAB, save_path='path_to_save')


### Map plot of A, B and R

In [ ]:
class ZR_test_dataset(Dataset):
    def __init__(self, csv_file):
        # Load the dataset
        df = pd.read_csv(csv_file)
        
        # It's assumed all these columns are numerical
        self.X = df[['radar_norm', 'dist_coast', 'dist_radar', 'topo', 'toy_sin', 'toy_cos', 'tod_sin', 'tod_cos']].values
        self.R = df['R'].values
        self.Z = df['Z'].values
        self.dBZ = df['dBZ'].values
        
        # Convert to PyTorch tensors once
        self.X = torch.tensor(self.X, dtype=torch.float)
        self.R = torch.tensor(self.R, dtype=torch.float)
        self.Z = torch.tensor(self.Z, dtype=torch.float)
        self.dBZ = torch.tensor(self.dBZ, dtype=torch.float)

        # Additional data like StationId and timestamp can be handled similarly if needed

    def __len__(self):
        return len(self.R)  # Assuming all columns have the same length

    def __getitem__(self, idx):
        # Directly index the pre-converted tensors
        return {'R': self.R[idx], 'Z': self.Z[idx], 'dBZ': self.dBZ[idx], 'x': self.X[idx]}


In [ ]:
# Load test data
test_dataset = ZR_test_dataset(csv_file='test_spatial.csv')
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model.eval()  # Set the model to evaluation mode

logAs = []  
Bs = []
logR_preds = []

with torch.no_grad(): 
    for batch in tqdm(test_dataloader):
        features = batch['x'].to(device)

        # Make predictions
        logA, B, x = model(features)

        # Collect predictions
        logAs.append(logA)
        Bs.append(B)
        logR_preds.append(x)

In [ ]:
# Load test data
df_test = pd.read_csv('test_spatial.csv')

# Convert to appropriate types or calculate fields for entire DataFrame
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test['year'] = df_test['timestamp'].dt.year
df_test['month'] = df_test['timestamp'].dt.month

# Apply calculations only for valid rows
df_test[['MP', 'MP_opt', 'R_pred_R', 'R_pred_AB']] = 0.0
df_test['MP'] = np.array(pf.dbz_to_R_marshall_palmer(df_test['dBZ'], 200, 1.6)).astype(float)
df_test['MP_opt'] = np.array(pf.dbz_to_R_marshall_palmer(df_test['dBZ'], 186, 3.8)).astype(float)
df_test['A'] = np.exp(torch.cat(logAs).cpu().numpy())
df_test['B'] = torch.cat(Bs).cpu().numpy()
df_test['R_pred_R'] = np.exp(torch.cat(logR_preds).cpu().numpy())
df_test['R_pred_AB'] = np.array(pf.dbz_to_R_marshall_palmer(df_test['dBZ'], df_test['A'], df_test['B']))

df_test.head()

In [ ]:
df_test.to_csv('df_results.csv')

In [ ]:
# Initialize the matrix with NaNs instead of zeros to avoid a second pass for replacement
R_pred_R = np.full((1728, 1984), np.nan)
R_pred_AB = np.full((1728, 1984), np.nan)
A_pred = np.full((1728, 1984), np.nan)
B_pred = np.full((1728, 1984), np.nan)
dBZ = np.full((1728, 1984), np.nan)

# Extract row and column indices and R_pred_R values
rows = df_test['row'].astype(int).values
cols = df_test['col'].astype(int).values
R_pred_values = df_test['R_pred_R'].values
R_pred_AB_values = df_test['R_pred_AB'].values
A_values = df_test['A'].values
B_values = df_test['B'].values
dBZ_values = df_test['dBZ'].values

# Use numpy advanced indexing to assign values directly
R_pred_R[rows, cols] = R_pred_values
R_pred_AB[rows, cols] = R_pred_AB_values
A_pred[rows, cols] = A_values
B_pred[rows, cols] = B_values
dBZ[rows, cols] = dBZ_values

In [ ]:
importlib.reload(pf)
from matplotlib import colors  

# Define your rain rate boundaries
rain_boundaries = [0.001, .5, 1, 2, 3, 4, 5, 7.5, 10, 15, 20]
  
cmap = colors.ListedColormap(["#85E3E4", '#42D8D8', '#42AFD8', '#4282D8', "#FFE600", '#FFAF00', '#FF5050', '#FF1A1A', "#BD0000", "#8C0000"])
boundaries = np.round(rain_boundaries)
norm = colors.BoundaryNorm(boundaries, cmap.N, clip=True)

save_path = param_string + 'NN_R_rain_map' + '.png'
pf.rain_map(R_pred_R, 'Rain Intensity', cmap = cmap, norm = norm, save_path=save_path)

In [ ]:
save_path = param_string + 'NN_AB_rain_map'  + '.png'
pf.rain_map(R_pred_AB, 'Rain Intensity', cmap = cmap, norm = norm, save_path=save_path)

In [ ]:
from matplotlib.colors import BoundaryNorm
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap

def read_rgb_file(file_path):
    with open(file_path, 'r') as file:
        # Read all lines in the file
        lines = file.readlines()

    # Skip lines that do not contain color information
    color_lines = [line for line in lines if line.strip() and not line.startswith('#') and not line.startswith('ncolors')]
    
    # Extract the RGB values
    rgb_values = [tuple(map(int, line.strip().split()[:3])) for line in color_lines]
    
    # Normalize the RGB values to the range [0, 1]
    rgb_normalized = [(r/255.0, g/255.0, b/255.0) for r, g, b in rgb_values]
    
    # Create a colormap from the RGB values
    colormap = LinearSegmentedColormap.from_list("custom_radar", rgb_normalized)
    
    return colormap

# Use the function to create the colormap
custom_colormap = read_rgb_file('radar.rgb')
boundaries = np.linspace(0, 75, 16)
norm = BoundaryNorm(boundaries, ncolors=custom_colormap.N, clip=True)

save_path = param_string + 'dBZ_rain_map' +  '.png'
pf.rain_map(dBZ, 'dBZ', cmap = custom_colormap , norm = norm, save_path=save_path)

In [ ]:
save_path = param_string + 'A_map' + '.png'
pf.rain_map(A_pred, 'A', cmap = 'turbo', norm = None, save_path=save_path)

In [ ]:
save_path =  param_string +'B_map'  + '.png'
pf.rain_map(B_pred, 'B', cmap = 'turbo', norm = None, save_path=save_path)

In [ ]:
torch.save(model.state_dict(),  param_string + 'model_weights.pth')